In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Light Automl Installation

In [1]:
!pip install -U lightautoml

     |████████████████████████████████| 262 kB 4.5 MB/s eta 0:00:01
     |████████████████████████████████| 204 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 232 kB 7.6 MB/s eta 0:00:01
     |████████████████████████████████| 424 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 14.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 100 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 1.2 MB/s  eta 0:00:01
     |████████████████████████████████| 84 kB 1.0 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 1.7 MB/s  eta 0:00:01
     |████████████████████████████████| 121 kB 31.2 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 4.0 MB/s  eta 0:00:01
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=c3c0a898613f8a873451bb8027e9fb6ec10e644191d5ce0e9a8a67acd398fb8e
  S

# Step 0.1. Import libraries

**1- Essential python DS libraries like numpy, pandas, scikit-learn and torch (the last we will use in the next cell)


**2- LightAutoML modules: presets for AutoML, task and report generation module****

In [2]:
# Standard python libraries
import os
import time

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error
import matplotlib.pyplot as plt
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.report.report_deco import ReportDeco

# Step 0.2. Constants

* N_THREADS
* N_FOLDS
* RANDOM_STATE
* TIMEOUT
* TARGET_NAME

In [3]:
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TIMEOUT = 1200
TARGET_NAME = 'target'

# Step 0.3.  models setup

In [4]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Step 0.4. Data loading

In [11]:
%%time

train_data = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
train_data.head()



CPU times: user 32.4 ms, sys: 2.81 ms, total: 35.2 ms
Wall time: 50.8 ms


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [12]:
train_data.shape

(7111, 12)

In [13]:
%%time

test_data = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
test_data.head()

CPU times: user 8.29 ms, sys: 2.01 ms, total: 10.3 ms
Wall time: 18.2 ms


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,2011-01-01 00:00:00,8.0,41.3,0.4375,1108.8,745.7,797.1,880.0,1273.1
1,2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714.0
2,2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878.0,693.7,941.9,1300.8
3,2011-01-01 03:00:00,5.0,52.3,0.4693,1139.7,916.2,725.6,1011.0,1283.0
4,2011-01-01 04:00:00,4.5,57.5,0.4650,1022.4,838.5,871.5,967.0,1142.3


In [14]:
test_data.shape

(2247, 9)

In [15]:
%%time

sample_sub = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')
sample_sub.head()

CPU times: user 5.66 ms, sys: 2.63 ms, total: 8.29 ms
Wall time: 16.3 ms


,date_time,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2011-01-01 00:00:00,2.0,10.0,300.0
1,2011-01-01 01:00:00,2.0,10.0,300.0
2,2011-01-01 02:00:00,2.0,10.0,300.0
3,2011-01-01 03:00:00,2.0,10.0,300.0
4,2011-01-01 04:00:00,2.0,10.0,300.0


In [16]:
sample_sub.shape

(2247, 4)

# model building

# Step 1. Task setup

we build LightAutoML model  that should solve with specific loss and metric if necessary 

In [17]:
%%time

def rmsle_metric(y_true, y_pred, **kwargs):
    return mean_squared_log_error(y_true, np.clip(y_pred, 0, None), **kwargs) ** 0.5

task = Task('reg', loss = 'rmsle', metric = rmsle_metric)

CPU times: user 2.91 ms, sys: 0 ns, total: 2.91 ms
Wall time: 2.64 ms


# Step 2. Feature roles setup
    we generate here a target rule first

In [18]:
?DatetimeRole

Init signature:
DatetimeRole(
    dtype: Union[Callable, type, str] = <class 'numpy.datetime64'>,
    seasonality: Union[Sequence[str], NoneType] = ('y', 'm', 'wd'),
    base_date: bool = False,
    date_format: Union[str, NoneType] = None,
    unit: Union[str, NoneType] = None,
    origin: Union[str, datetime.datetime] = 'unix',
    force_input: bool = False,
    base_feats: bool = True,
    country: Union[str, NoneType] = None,
    prov: Union[str, NoneType] = None,
    state: Union[str, NoneType] = None,
)
Docstring:      Datetime role.
Init docstring:
Create datetime role with specific dtype and attrs.

Args:
    dtype: Variable type.
    seasonality: Seasons to extract from date.
      Valid are: 'y', 'm', 'd', 'wd', 'hour',
      'min', 'sec', 'ms', 'ns'.
    base_date: Base date is used to calculate difference
      with other dates, like `age = report_dt - birth_dt`.
    date_format: Format to parse date.
    unit: The unit of the arg denote the unit, pandas like, see more:
   

In [19]:
%%time

targets_and_drop = {
    'target_carbon_monoxide': ['target_benzene', 'target_nitrogen_oxides'],
    'target_benzene': ['target_carbon_monoxide', 'target_nitrogen_oxides'],
    'target_nitrogen_oxides': ['target_carbon_monoxide', 'target_benzene']
}

roles = {
    DatetimeRole(base_date=False, base_feats=True, seasonality=('d', 'wd', 'hour')): 'date_time'
}

CPU times: user 105 µs, sys: 0 ns, total: 105 µs
Wall time: 110 µs


# Step 3. LightAutoML model creation - TabularAutoML preset

**we are going to create LightAutoML model with TabularAutoML class - preset with default model structure**

In [ ]:
Step 4. Create submission file

# task - the type of the ML task (the only must have parameter)

**timeout **- time limit in seconds for model to train

**cpu_limit** - vCPU count for model to use

**reader_params** - parameter change for Reader object inside preset, which works on the first step of data preparation: automatic 

**feature typization**, preliminary almost-constant features, correct CV setup etc. For example, we setup n_jobs threads for 

**typization algo**, cv folds and random_state as inside CV seed.

**general_params** - we use use_algos key to setup the model structure to work with (two LGBM models and two CatBoost models on the first level and their weighted composition creation on the second). This setup is only to speedup the kernel, you can remove this general_params setup if you want the whole LightAutoML model to run.

In [ ]:
%%time 
importances = {}
dt = pd.to_datetime(train_data['date_time'])
for targ in targets_and_drop:
    print('='*50, '='*50, sep = '\n')
    automl = TabularAutoML(task = task, 
                           timeout = TIMEOUT,
                           cpu_limit = N_THREADS,
                           reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                           general_params={'use_algos': [['lgb', 'lgb_tuned', 'cb', 'cb_tuned']]}
                          )

    roles['target'] = targ
    roles['drop'] = targets_and_drop[targ]
    
    if targ == 'target_nitrogen_oxides':
        oof_pred = automl.fit_predict(train_data[dt >= np.datetime64('2010-09-01')], roles = roles)
        else:
        oof_pred = automl.fit_predict(train_data, roles = roles)
    print('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))
    
    # Fast feature importances calculation
    fast_fi = automl.get_feature_scores('fast')
    importances[targ] = fast_fi
    
    test_pred = automl.predict(test_data)
    print('Prediction for te_data:\n{}\nShape = {}'.format(test_pred, test_pred.shape))
    
    sample_sub[targ] = np.clip(test_pred.data[:, 0], 0, None)

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


Start automl preset with listed constraints:
- time: 1200 seconds
- cpus: 4 cores
- memory: 16 gb

Train data shape: (7111, 12)
Feats was rejected during automatic roles guess: []


Layer 1 ...
Train process start. Time left 1168.9333124160767 secs
Start fitting Selector_LightGBM ...

===== Start working with fold 0 for Selector_LightGBM =====

Training until validation scores don't improve for 200 rounds
[100]	valid's l2: 0.0439204	valid's Opt metric: 0.209572
[200]	valid's l2: 0.0204068	valid's Opt metric: 0.142853
[300]	valid's l2: 0.0156261	valid's Opt metric: 0.125004
[400]	valid's l2: 0.013688	valid's Opt metric: 0.116996
[500]	valid's l2: 0.0127418	valid's Opt metric: 0.112879
[600]	valid's l2: 0.0122053	valid's Opt metric: 0.110478
[700]	valid's l2: 0.0118744	valid's Opt metric: 0.10897
[800]	valid's l2: 0.0116491	valid's Opt metric: 0.107931
[900]	valid's l2: 0.0114664	valid's Opt metric: 0.107081
[1000]	valid's l2: 0.0113301	valid's Opt metric: 0.106443
[1100]	valid's l2: 0.0

In [ ]:
for targ in targets_and_drop:
    plt.figure(figsize = (20, 10))
    importances[targ].set_index('Feature')['Importance'].plot.bar()
    plt.title('Feature importances for {} model'.format(targ))
    plt.grid(True)
    plt.show()

# Step 4. Create submission file

In [ ]:
sample_sub

In [ ]:
sample_sub.to_csv('lightautoml_more_algs_without_month_feat.csv', index = False)

# Additional materials

# * Official LightAutoML github repo
* LightAutoML documentation